# Resource estimation profiling in quantum adders

In this sample, we are implementing two quantum adders and then inspecting them
using the profiling feature in the Azure Quantum Resource Estimator.  The
profiling feature allows you to analyze how sub-operations in the quantum
algorithm impact the overall resources.

In particular, you will learn how to use the `profiling.call_stack_depth` and
`profiling.inline_functions` job parameters to enable profiles in resource
estimation jobs, and how to use the `call_graph` and `profile` properties on
resource estimation results to show call graphs and resource estimation
profiles, respectively.

The notebook is structured as follows.  First, we connect to an Azure Quantum
workspace and import the necessary functions, then we describe the two quantum
adder implementations.  It's okay to skip the implementation part and jump right
into the next session, which shows how to use the profiling feature to construct
call graphs and resource estimation profiles for time and space.  Finally, we
show how the detailed profiling information can be used for advanced analyses.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator
import pandas
import qsharp
import re

In [ ]:
workspace = Workspace (
    resource_id = "",
    location = ""
)

## Quantum adders

In this section, we implement two Q# operations `RippleCarryAdd` and
`LookAheadAdd` together with estimation entry points `EstimateRippleCarryAdd`
and `EstimateLookAheadAdd`, respectively, which each can be provided a
bit-width.  Feel free to skip over the implementation details of these adders,
right to the next section in which we are using the profiling feature to analyze
them.

In the realm of quantum computing, the ability to perform addition is crucial
for a wide range of applications. In this section, we will introduce two
different quantum adders that can be used to perform addition: the ripple-carry
adder and the carry-lookahead adder. Given two qubit $n$-bit registers
$|x\rangle = |(x_{n-1}\dots x_1x_0)_2\rangle$ and $|y\rangle = |(y_{n-1}\dots
y_1y_0)_2\rangle$, the goal is compute an $n+1$ bit register $|z\rangle = |x +
y\rangle$.

Both implementations will also support the overflowing variant, in which the
output register has $n$ bits, and we compute $|z\rangle = |(x + y) \bmod
2^n\rangle$.

### Ripple-carry adder

We review the ripple-carry adder described in [Halving the cost of quantum
addition](https://arxiv.org/abs/1709.06648).  We focus on the out-of-place
variant, in which the sum of input registers $|x\rangle$ and $|y\rangle$ is
computed into a $|0\rangle$-initialized output register $|z\rangle$.  A
ripple-carry adder adds the bits of $|x\rangle$ and $|y\rangle$ one by one,
starting from the rightmost bit, and propagates the carry generated by the
addition to the next bit to the left. This process is repeated until all the
bits have been added, resulting in a final sum and carry-out.  The core
component of the ripple-carry adder is the full adder, which is a one-bit adder
that takes a carry input bit, and returns a carry output bit, in addition to the
sum bit.

In [ ]:
%%qsharp
internal operation FullAdder(carryIn : Qubit, x : Qubit, y : Qubit, carryOut : Qubit) : Unit is Adj {
    CNOT(x, y);
    CNOT(x, carryIn);
    ApplyAnd(y, carryIn, carryOut);
    CNOT(x, y);
    CNOT(x, carryOut);
    CNOT(y, carryIn);
}

With this operation at hand, implementing the ripple carry adder is as simple as
carrying the carry bit through successive full adder invocations.  The two CNOT
operations at the end handle the case in which $|z\rangle$ has $n$ bits, in
which the sum for the most significant bit is not computed by a full adder
(since the corresponding carry out bit does not exist).

In [ ]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation RippleCarryAdd(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj {
    let n = Length(xs);
    let m = Length(zs);

    Fact(Length(ys) == n, "Registers xs and ys must be of same length");
    Fact(m == n or m == n + 1, "Register zs must be same length as xs or one bit larger");

    for k in 0..m - 2 {
        FullAdder(zs[k], xs[k], ys[k], zs[k + 1]);
    }

    if n > 0 and n == Length(zs) {
        CNOT(Tail(xs), Tail(zs));
        CNOT(Tail(ys), Tail(zs));
    }
}

Finally, we are creating an entry point for resource estimation, in which we are
calling the carry-lookahead adder for a given bitwidth, which can be provided as
an input argument. (We are considering the $n+1$ output register in this
notebook.)

In [ ]:
%%qsharp
operation EstimateRippleCarryAdd(bitwidth : Int) : Unit {
    use xs = Qubit[bitwidth];
    use ys = Qubit[bitwidth];
    use zs = Qubit[bitwidth + 1];

    RippleCarryAdd(xs, ys, zs);
}


### Carry-lookahead adder

In [_A logarithmic-depth quantum carry-lookahead adder_](https://arxiv.org/abs/quant-ph/0406142), the authors present an out-of-place quantum adder implementation based on the [classical carry-lookahead adder method](https://en.wikipedia.org/wiki/Carry-lookahead_adder).  To briefly summarize, the idea of carry-lookahead adder is to compute all carry bits $c_i$ based on propagate bits $p_i = x_i \oplus y_i$ and generate bits $g_i = x_i \land y_i$, without requiring other carry bits except for the carry-in $c_0$.

For example, the first carry bit can be computed as $c_1 = g_0 \oplus (p_0 \land c_0)$, since either it is generated from bits $x_0$ and $y_0$ (when both are 1, and therefore $g_0 = 1$) or the carry bit $c_0$ is propagated (if either $x_0$ or $y_0$ is 1, and therefore $p_0 = 1$).  More significant carry bits are computed in a similar way, for example $c_3 = g_2 \oplus (g_1 \land p_2) \oplus (g_0 \land p_1 \land p_2) \oplus (c_0 \land p_0 \land p_1 \land p_2)$.  That is, $c_3$ is either generated from bits at index 2, or generated from bits at index 1 _and_ propagated from bits at index 2, and so on.

In order to minimize AND gates, these intermediate products can be computed in a clever way, as well as in logarithmic depth.  We are now looking at an implementation of the carry-lookahead adder in Q#, and start by implementing a helper function to compute the number of 1-bits an an integer, also called Hamming weight, using a compact implementation based on a sequence of bitwise manipulations (you can learn more about these constants and why it works [on this article in Wikipedia](https://en.wikipedia.org/wiki/Hamming_weight#Efficient_implementation)).

In [ ]:
%%qsharp
function HammingWeight(n : Int) : Int {
    mutable i = n - ((n >>> 1) &&& 0x5555555555555555);
    set i = (i &&& 0x3333333333333333) + ((i >>> 2) &&& 0x3333333333333333);
    return (((i + (i >>> 4)) &&& 0xF0F0F0F0F0F0F0F) * 0x101010101010101) >>> 56;
}

Next we are going to implement internal routines that compute the generalized propagate bits, generalized generate, as well as the carry bits from them.  These are called `PRounds`, `GRounds`, and `CRounds`, and descriptions of their implementations can be found in the paper above.

In [ ]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

internal operation PRounds(pWorkspace : Qubit[][]) : Unit is Adj {
    for ws in Windows(2, pWorkspace) {
        let (current, next) = (Rest(ws[0]), ws[1]);

        for (m, target) in Enumerated(next) {
            ApplyAnd(current[2 * m], current[2 * m + 1], target);
        }
    }
}

internal operation GRounds(pWorkspace : Qubit[][], gs : Qubit[]) : Unit is Adj {
    let numRounds = Length(pWorkspace);
    let n = Length(gs);

    for t in 1..numRounds {
        let length = Floor(IntAsDouble(n) / IntAsDouble(2^t)) - 1;
        let ps = pWorkspace[t - 1][0..2...];

        for m in 0..length {
            CCNOT(gs[2^t * m + 2^(t - 1) - 1], ps[m], gs[2^t * m + 2^t - 1]);
        }
    }
}

internal operation CRounds(pWorkspace : Qubit[][], gs : Qubit[]) : Unit is Adj {
    let n = Length(gs);

    let start = Floor(Lg(IntAsDouble(2 * n) / 3.0));
    for t in start..-1..1 {
        let length = Floor(IntAsDouble(n - 2^(t - 1)) / IntAsDouble(2^t));
        let ps = pWorkspace[t - 1][1..2...];

        for m in 1..length {
            CCNOT(gs[2^t * m - 1], ps[m - 1], gs[2^t * m + 2^(t - 1) - 1]);
        }
    }
}

With these operations we can build an operation that computes the carry bits
from the initial propagate and generate bits.  Note that the generalized
propagate bits are computed out-of-place into some helper qubits `qs`, whereas
the generalized generate and carry bits are computed in-place into the register
`gs`, which contains the initial generate bits.

In [ ]:
%%qsharp
internal operation ComputeCarries(ps : Qubit[], gs : Qubit[]) : Unit is Adj {
    let n = Length(gs);

    let numRounds = Floor(Lg(IntAsDouble(n)));
    use qs = Qubit[n - HammingWeight(n) - numRounds];

    let registerPartition = MappedOverRange(t -> Floor(IntAsDouble(n) / IntAsDouble(2^t)) - 1, 1..numRounds - 1);
    let pWorkspace = [ps] + Partitioned(registerPartition, qs);

    within {
        PRounds(pWorkspace);
    } apply {
        GRounds(pWorkspace, gs);
        CRounds(pWorkspace, gs);
    }
}

Now, we are all set up to implement the carry-lookahead adder, by first
computing the initial propagate and generate bits, then computing the carry
bits, and finally computing the output bits using the sums (initial propagate
bits) together with the carry bits.  Note that this implementation supports both
an variant where the output register is 1 bit larger and does not overflow, as
well as a variant in which the sum is computed modulo $2^n$.  The latter uses
the former by using a special treatment of the most-significant bits of the
input registers.

In [ ]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;

operation LookAheadAdd(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj {
    let n = Length(xs);
    let m = Length(zs);

    Fact(Length(ys) == n, "Registers xs and ys must be of same length");
    Fact(m == n or m == n + 1, "Register zs must be same length as xs or one bit larger");

    if m == n + 1 { // with carry-out
        // compute initial generate values
        for k in 0..n - 1 {
            ApplyAnd(xs[k], ys[k], zs[k + 1]);
        }

        within {
            // compute initial propagate values
            ApplyToEachA(CNOT, Zipped(xs, ys));
        } apply {
            if n > 1 {
                ComputeCarries(Rest(ys), Rest(zs));
            }

            // compute sum into carries
            for k in 0..n - 1 {
                CNOT(ys[k], zs[k]);
            }
        }
    } else { // without carry-out
        LookAheadAdd(Most(xs), Most(ys), zs);
        CNOT(Tail(xs), Tail(zs));
        CNOT(Tail(ys), Tail(zs));
    }
}

Finally, we are creating an entry point for resource estimation, in which we are
calling the carry-lookahead adder for a given bitwidth, which can be provided as
an input argument.

In [ ]:
%%qsharp
operation EstimateLookAheadAdd(bitwidth : Int) : Unit {
    use xs = Qubit[bitwidth];
    use ys = Qubit[bitwidth];
    use zs = Qubit[bitwidth + 1];

    LookAheadAdd(xs, ys, zs);
}


## Profiling

With these two adder implementations, we are now ready for some profiling.  The
profiling feature in Azure Quantum Resource Estimator will create a resource
estimation profile that will show how sub operations in the program (e.g.,
`FullAdder` inside `RippleCarryAdd`) contribute to the overall costs.

There are two important concepts to review to best understand the outputs of the
profiling feature, a *call graph* and a *call tree*.  The *call graph* is static
representation of the quantum program which informs which operations call which
other operations.  For example, `RippleCarryAdder` calls `FullAdder`, but both
of these operations call `CNOT`.  The call graph contains a node for each
operation and a directed edge for each calling relation.  The call graph may
contain cycles, e.g., in the case of recursive operations.

In contrast, a call tree is a dynamic representation of the program execution in
which there are no cycles and for each node there is a clear path from the root
node.  For example, distinguishes the calls to `CCNOT` from `GRounds` and
`CRounds` within the `ComputeCarries` operation in the carry-lookahead adder.

But let's start looking at concrete examples, and create a resource estimator
instance together with a parameter object to configure the bitwidth argument and
enable the profiling feature.

In [ ]:
estimator = MicrosoftEstimator(workspace)

params = estimator.make_params()
params.arguments["bitwidth"] = 32
params.profiling.call_stack_depth = 10

We enable the profiling feature by setting the `call_stack_depth` variable in
the `profiling` group to a number that indicates the maximum level up to which
sub operations are tracked.  More precisely, the entry point operation is at
level 0.  Any operation called from the entry point is at level 1, any operation
therein at 2, and so on.  The call stack depth is setting a maximum value to an
operation's level in the call stack for which we track resources in the profile.

Next, we submit a resource estimation job for the ripple carry adder by
providing the Q# operation `EstimateRippleCarryAdd` and the job parameter
object.  We store there the results of the job in the variable `result_rca`,
where RCA is an abbreviation for ripple-carry adder.

In [ ]:
job = estimator.submit(EstimateRippleCarryAdd, input_params=params)
result_rca = job.get_results()

We can inspect the call graph by calling the `call_graph` property on the result
object.  It displays the call graph with the node corresponding to the entry
point operation at the top and aligns other operations top-down according to
their level.

In [ ]:
result_rca.call_graph

Note that the operation names are mangled, i.e., they contain additional
information.  In this case, this can be a prefix `SNIPPET` or `ENTRYPOINT`,
which is generated by `qsharp` Python library from the Q# that is integrated
into the notebook.  Some operations are prefixed by their namespace, and
operations have a suffix to indicate their variant (e.g., `body` for their
`body` implementation, and `adj` for their `adjoint` implementation).

As described above, we can see that `RippleCarryAdd` calls both the `FullAdder`
and `CNOT`, and that the `FullAdder` also calls `CNOT` and `ApplyAnd`.  In fact,
the `CNOT` operation is called from 3 other operations.

Next, let's also generate resource estimates for the carry-lookahead
implementation.

In [ ]:
job = estimator.submit(EstimateLookAheadAdd, input_params=params)
result_cla = job.get_results()

Again, we can look at its call graph.

In [ ]:
result_cla.call_graph

This call graph is much larger since more operations are involved.  We can see
the `PRounds`, `GRounds`, and `CRounds` operations, and see that `GRounds` and
`CRounds` call `CCNOT`, but `PRounds` calls `ApplyAnd`.  We also see that the
`adjoint` variant of `PRounds` calls the `adjoint` variant of `ApplyAnd`.

It's possible to obtain a more compact call graph (and resource estimation
profile) by inlining some functions, e.g., those that just call a different
function and have no other logic inside.  We can use the `inline_functions`
parameter for that:

In [ ]:
params.profiling.inline_functions = True
job = estimator.submit(EstimateLookAheadAdd, input_params=params)
result_cla_inline = job.get_results()
result_cla_inline.call_graph

Although the call graph is now smaller, certain details—such as the rounds in
`ComputeCarries`—have been inlined and are no longer available for analysis.
Thus, the value for the `inline` parameter is typically chosen based on the
desired type of analysis. For the purposes of our remaining analysis, we will be
considering profiles for the adder operations without inlining.

Next we are inspecting the resource estimation profiles of both results.  These
will provide the sub operations' contributions to runtime, logical depth,
physical qubits for the algorithm, and logical qubits for the algorithm.  Let's
first have an overview of the total counts for each result:

In [ ]:
breakdown_rca = result_rca['physicalCounts']['breakdown']
breakdown_cla = result_cla['physicalCounts']['breakdown']

pandas.DataFrame(data = {
    "Runtime": [f"{result_rca['physicalCounts']['runtime']} ns", f"{result_cla['physicalCounts']['runtime']} ns"],
    "Logical depth": [breakdown_rca['logicalDepth'], breakdown_cla['logicalDepth']],
    "Physical qubits": [breakdown_rca['physicalQubitsForAlgorithm'], breakdown_cla['physicalQubitsForAlgorithm']],
    "Logical qubits": [breakdown_rca['algorithmicLogicalQubits'], breakdown_cla['algorithmicLogicalQubits']],
}, index = ["Ripple-carry adder", "Carry-lookahead adder"])

The resource estimation profile is generated as a JSON file that can be read
using the speedscope interactive online profile viewer.  In order to generate
and download the file call the `profile` property on a result object.

In [ ]:
result_rca.profile

After downloading and opening the profile, we first see the runtime profile. For
this ripple-carry adder, we can readily see that all the runtime cost is caused
by the And operation inside the full adder.  No other operation contributes cost
the overall runtime.  The top most box is the entry point operation and the root
note of the call tree.  In the next layer below are all operations that are
called from the root node and that contribute to the overall runtime.

<center><img src="https://raw.githubusercontent.com/microsoft/Quantum/main/samples/azure-quantum/resource-estimation/profile_rca_runtime.png" style="width: 910px" /></center>

On the top center is a button that displays _Runtime (1/4)_, which we can press
to look at profiles for the other metrics.  If we click on physical algorithmic
qubits, we get this profile:

<center><img src="https://raw.githubusercontent.com/microsoft/Quantum/main/samples/azure-quantum/resource-estimation/profile_rca_qubits.png" style="width: 910px" /></center>

For the number of qubits we account how many new qubits were allocated by some
operation and track the maximum number of allocated qubits.  In this sample,
`EstimateRippleCarryAdd` allocated all qubits for the adder and there are no
additional helper qubits, e.g., in the implementation of `FullAdder`.  The entry
point operation accounts for the additional qubits that are required for the
padding in the 2D layout on the surface code.

Let's also generate the profile for the carry-lookahead adder.

In [ ]:
result_cla.profile

The runtime profile for the carry-lookahead adder contains more operations:

<center><img src="https://raw.githubusercontent.com/microsoft/Quantum/main/samples/azure-quantum/resource-estimation/profile_cla_runtime.png" style="width: 910px" /></center>

We can see that the AND gates in the `LookAheadAdd` operation and the
`ComputeCarries` operation contribute to the overall runtime.  Inside the
`ComputeCarries` operation, we can analyze the contribution of the sub
operations for the different rounds, and, e.g., note that the `adjoint` variant
for the `PRounds` takes the fewest time, whereas all other 3 rounds are of
similar complexity.

In the qubit profile, we see that some additional helper qubits are allocated in
the `ComputeCarries` operation to hold the auxiliary $p$ bits, that are computed
out-of-place and required in the computation of the `GRounds` and `CRounds`.

<center><img src="https://raw.githubusercontent.com/microsoft/Quantum/main/samples/azure-quantum/resource-estimation/profile_cla_qubits.png" style="width: 910px" /></center>

## Advanced analysis

In this notebook's final section, we will be conducting advanced analysis on the
profiling data. Specifically, we will programmatically access the profiling data
and examine the impact of `PRounds`, `GRounds`, and `CRounds` on the
carry-lookahead adder's runtime for increasing bitwidths.

To begin, we will establish parameters for a batching job with bitwidths ranging
from $2^1 = 2$ to $2^{10} = 1024$, with power-of-2 steps. We will also set the
call stack depth to 10, as demonstrated in the examples above. It's worth noting
that while the call stack depth parameter applies to all items in the batching
parameters, the bitwidth must be specified for each item individually.

In [ ]:
bitwidths = [2**k for k in range(1, 11)]
params = estimator.make_params(num_items=len(bitwidths))

params.profiling.call_stack_depth = 10
for i, bitwidth in enumerate(bitwidths):
    params.items[i].arguments['bitwidth'] = bitwidth

We are submitting the job and store its results in `results_all`.

In [ ]:
job = estimator.submit(EstimateLookAheadAdd, input_params=params)
results_all = job.get_results()

The raw profiling data can be accessed in JSON format via the `'profile'` key in
a resource estimation result. For example, to access the profiling data for the
first item in `results_all`, we would use `results_all[0]['profile']`. The
format follows a custom scheme for speedscope and is documented in detail
[here](https://github.com/jlfwong/speedscope/blob/master/src/lib/file-format-spec.ts),
with the corresponding JSON schema available
[here](https://www.speedscope.app/file-format-schema.json).
   
Each node in the tree is assigned a frame with an index, and the profile
contains samples organized by calling order, with each sample assigned a weight
(e.g. runtime). To locate the frame associated with a given round name (e.g.
`PRounds`), the following Python function can be used: it finds the frame, then
identifies all samples that contain it, and sums up the corresponding weights.

In [ ]:
def rounds_runtime(profile, round_name):
    # Find the frame index which name contains `round_name` and ends in "body".
    frame_index = [i for (i, frame) in enumerate(profile['shared']['frames']) if re.match(f'.*{round_name}.+body', frame['name'])][0]

    # The runtime profile is the first profile
    runtime_profile = profile['profiles'][0]

    # Get variables to the samples and weights field of the runtime profile
    samples = runtime_profile['samples']
    weights = runtime_profile['weights']

    # Sum up all the weights that correspond to samples that contain the operation,
    # i.e., that contain the frame_index
    return sum(weight for (sample, weight) in zip(samples, weights) if frame_index in sample)

We now extract the profile for each job result item and use the `rounds_runtime`
function to obtain the runtime for each round, add it to a data frame together
with the total runtime and return a plot.

In [ ]:
entries = []
for idx in range(len(results_all)):
    bitwidth = bitwidths[idx]
    result = results_all[idx]
    profile = result['profile']

    total_runtime = result['physicalCounts']['runtime']
    entries.append([
        rounds_runtime(profile, "PRounds"),
        rounds_runtime(profile, "GRounds"),
        rounds_runtime(profile, "CRounds"),
        total_runtime
    ])

df = pandas.DataFrame(data=entries, index=bitwidths, columns=["PRounds", "GRounds", "CRounds", "Total"])
ax = df.plot(logx=True, xticks=bitwidths, xlabel="Bitwidth", ylabel="Runtime (ns)")
# show all xticks
from matplotlib.text import Text
ax.set_xticklabels([Text(b, 0, b) for b in bitwidths]);

Note how the total runtime grows much faster compared to the runtime of the
rounds.  The reason is that we need $O(n)$ AND gates in the preparation part of
`LookAheadAdd` but only $O(\log(n))$ AND and CCNOT gates in the `ComputeCarries`
operation.

Further note that logical depth of a the carry-lookahead adder is also
logarithmic in $n$, since on the logical level, all AND and CCNOT gates, in both
the preparation parts and in the rounds can be applied in parallel.  However,
when mapping to surface code operations using Parallel Synthesis Sequential
Pauli Computation (PSSPC), these operations are sequentialized (see Appendix D
in [Assessing requirements to scale to practical quantum
advantage](https://arxiv.org/pdf/2211.07629.pdf))

## Next steps

Great job! You now know how to use the profiling feature of the Azure Quantum
Resource Estimator to analyze how different parts of your program contribute to
overall resource estimates.

To summarize, you can use the use the `profiling.call_stack_depth` and
`profiling.inline_functions` job parameters to enable profiles in resource
estimation jobs.  In the resource estimation results that you receive after
successful job submission, the `call_graph` and `profile` properties show call
graphs and resource estimation profiles, respectively.

We encourage you to further explore this feature and try out the following
ideas:
   
* Experiment with changing the `call_stack_depth` parameter
* Investigate call graphs and profiles of recursive programs
* Generate profiles from programs in other notebooks
* Perform an advanced profile analysis to compare the number of helper qubits to
  the number of input and output qubits in the carry-lookahead implementation